# Data acquisition in arbok

### Arbok module imports

In [1]:
from arbok.core.program import Program
from arbok.core.sample import Sample

2023-09-18 14:36:22,230 - qm - INFO     - Starting session: aecef8f5-346e-4e2d-8719-6545473cad48


In [2]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.control.adb import ADB
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

Instanciating sample and measurement objects

In [3]:
aurora = Sample('aurora', aurora_qua_config)
ESR_search = Program('ESR_search', sample = aurora)

In [4]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.control.adb import ADB
from arbok.samples.aurora.control.rabi import Rabi
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

In [5]:
from quantify_core.visualization import InstrumentMonitor

In [6]:
instrument_monitor = InstrumentMonitor("instrument_monitor")

Creating Subsequences and adding them to the measurement object `ESR_search`

In [7]:
init6 = InitOdd6dot('init6')
adb6 = ADB('adb6')
read6 = PSB6dotRead('read6')

In [8]:
ESR_search.add_subsequence(init6)
ESR_search.add_subsequence(adb6)
ESR_search.add_subsequence(read6)

In [9]:
#ESR_search.adb6.run_remote_simulation(duration=500)

In [10]:
ESR_search.adb6.IfQ1(5e6)

# Running the program in the quantify framework

In [11]:
import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl
from quantify_core.visualization.pyqt_plotmon import PlotMonitor_pyqt

In [12]:
#instrument_monitor = InstrumentMonitor("instrument_monitor")

Specifying the path for the database containing results

In [13]:
dh.set_datadir()

Data will be saved in:
C:\Users\labuser\quantify-data


Next we connect the physical OPX+ instrument to our program ... 

In [14]:
ESR_search.connect_OPX(host_ip= '192.168.0.3')

2023-09-18 14:36:25,320 - qm - INFO     - Performing health check
2023-09-18 14:36:25,377 - qm - INFO     - Health check passed
2023-09-18 14:36:28,826 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing existing identical Quantum Machine qm-1695012224766
2023-09-18 14:36:28,827 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1695012224766 is canceling running job 1684735462182.


Instanciating `MeasurementControl`

In [15]:
meas_ctrl = MeasurementControl('meas_ctrl')

Creating and connecting a `PlotMonitor` to our `MeasurementControl`

In [16]:
plotmon = PlotMonitor_pyqt("plotmon")
meas_ctrl.instr_plotmon(plotmon.name)

C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots is deprecated. Use "qcodes_loop.plots" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots.colors is deprecated. Use "qcodes_loop.plots.colors" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\qcodes\plots\pyqtgraph.py:17: QCoDeSDeprecationWarning: The qcodes.utils.qt_helpers module is deprecated. Use "qcodes_loop.utils.qt_helpers" as an alternative.
  import qcodes.utils.qt_helpers
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:19: QCoDeSDeprecationWarning: The qcodes.plots.pyqtgraph is deprecated. Use "qcodes_loop.plots

## How to measure: adiabatic ESR search

So far so good, now fully set up our adiabatic ESR search. We have a init, control and readout sequence. Now we have to define parameters to sweep and to measure.

Defining the frequency sweep list:

In [17]:
import numpy as np

In [18]:
f_detune=np.arange(25e6,80e6,0.1e6, dtype = int)

Defining the parameters to be swept and adding setpoints to them. Also parameters to be recorded are added. In this case the measured state of SET 1 and 2

In [19]:
ESR_search.settables = [
    ESR_search.adb6.IfQ1]
ESR_search.setpoints_grid = [
    f_detune]
ESR_search.gettables = [
    ESR_search.read6.diff_set2_state]

Just defined values are automatically added to `meas_ctrl` by running `prepare_meas_ctrl`

In [20]:
ESR_search.prepare_meas_ctrl(meas_ctrl)

... and run the job! Dont worry, the program will only run the first iteration and wait for us to retreive the result and only continue after that.

In [21]:
#ESR_search.run()

If all this is done, the actual measurement can be run! Check the taskbar for your plot monitor and see the buffered results come in.

In [22]:
#meas_ctrl.run("adb_ESR_search", soft_avg= 20)

## How to measure: Rabi Chevron

Sometimes we want to measure two dimensional parameter sweeps. This works very similarly to the 1D case. We just have to add a second settable a corresponding value array to the setpoints.

But lets first create the sequence:

In [23]:
meas_ctrl2 = MeasurementControl('meas_ctrl2')

In [24]:
plotmon2 = PlotMonitor_pyqt("plotmon2")
meas_ctrl2.instr_plotmon(plotmon2.name)

C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots is deprecated. Use "qcodes_loop.plots" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots.colors is deprecated. Use "qcodes_loop.plots.colors" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\qcodes\plots\pyqtgraph.py:17: QCoDeSDeprecationWarning: The qcodes.utils.qt_helpers module is deprecated. Use "qcodes_loop.utils.qt_helpers" as an alternative.
  import qcodes.utils.qt_helpers
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:19: QCoDeSDeprecationWarning: The qcodes.plots.pyqtgraph is deprecated. Use "qcodes_loop.plots

In [25]:
RabiChevron = Program('RabiChevron', sample = aurora)
rabi6 = Rabi('rabi6')
RabiChevron.add_subsequence(init6)
RabiChevron.add_subsequence(rabi6)
RabiChevron.add_subsequence(read6)

In [26]:
f_detune=np.arange(32.8e6,34.2e6,0.05e6, dtype = int)
print(len(f_detune))
t_list = np.arange(4,30e3/4,200, dtype = int)
print(len(t_list))

28
38


In [27]:
RabiChevron.settables = [
    RabiChevron.rabi6.IfQ1,
    RabiChevron.rabi6.tRabi]
RabiChevron.setpoints_grid = [
    f_detune,
    t_list]
RabiChevron.gettables = [
    RabiChevron.read6.diff_set1_state,
    RabiChevron.read6.diff_set2_state]

RabiChevron.connect_OPX(host_ip= '192.168.0.3')
RabiChevron.prepare_meas_ctrl(meas_ctrl2)

2023-09-18 14:36:32,771 - qm - INFO     - Performing health check
2023-09-18 14:36:32,848 - qm - INFO     - Health check passed


In [28]:
RabiChevron.run()

Adding qua <class 'int'> variable for IfQ1
Adding qua <class 'int'> variable for tRabi
Adding qua sweep loop for tRabi
Adding qua sweep loop for IfQ1
2023-09-18 14:36:36,589 - qm - INFO     - Sending program to QOP for compilation
2023-09-18 14:36:48,322 - qm - INFO     - Executing program


In [29]:
meas_ctrl2.run("RabiChevron", soft_avg=100)

Starting batched measurement...
Iterative settable(s) [outer loop(s)]:
	 --- (None) --- 
Batched settable(s):
	 IfQ1, tRabi 
Batch size limit: 1064

FETC 390 1064 diff_set1_state
WAIT 390 1064
WAIT 450 1064
WAIT 739 1064
WAIT 941 1064
READ 1064 1064
2023-09-18 14:36:49,628 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1064,)
FETC 1064 1064 diff_set2_state
READ 1064 1064
2023-09-18 14:36:50,203 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1064,)
  1% completed | elapsed time:      1s | time left:    177s  last batch size:   1064  FETC 1064 2128 diff_set1_state
WAIT 1064 2128
WAIT 1255 2128
WAIT 1406 2128
WAIT 1551 2128
WAIT 1689 2128
WAIT 1826 2128
WAIT 1951 2128
WAIT 2086 2128
READ 2128 2128
2023-09-18 14:36:51,877 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1064,)
FETC 2128 2128 diff_set2_state
READ 2128 2128
2023-09-18 14:36:52,288 - qm - WARNING  - Fetc

<xarray.Dataset>
Dimensions:  (dim_0: 1064)
Coordinates:
    x0       (dim_0) int32 32800000 32850000 32900000 ... 34100000 34150000
    x1       (dim_0) int32 4 4 4 4 4 4 4 ... 7404 7404 7404 7404 7404 7404 7404
Dimensions without coordinates: dim_0
Data variables:
    y0       (dim_0) float64 0.98 0.98 0.96 0.98 0.96 ... 0.99 0.99 0.99 1.0
    y1       (dim_0) float64 0.03 0.04 0.0 0.04 0.0 ... 0.17 0.14 0.16 0.12 0.08
Attributes:
    tuid:                             20230918-143648-625-d1f6ea
    name:                             RabiChevron
    grid_2d:                          True
    grid_2d_uniformly_spaced:         True
    1d_2_settables_uniformly_spaced:  False
    xlen:                             28
    ylen:                             38